In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

c:\Users\NonsoDev\Documents\Allcodes\Projects_DL_for resume\Recommender systems\book reccomender - llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\NonsoDev\.cache\kagglehub\datasets\dylanjcastillo\7k-books-with-metadata\versions\3


In [3]:
import pandas as pd
data = pd.read_csv(path + "/books.csv")
print("Number of books:", len(data))

Number of books: 6810


In [4]:
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6810 non-null   int64  
 1   isbn10          6810 non-null   object 
 2   title           6810 non-null   object 
 3   subtitle        2381 non-null   object 
 4   authors         6738 non-null   object 
 5   categories      6711 non-null   object 
 6   thumbnail       6481 non-null   object 
 7   description     6548 non-null   object 
 8   published_year  6804 non-null   float64
 9   average_rating  6767 non-null   float64
 10  num_pages       6767 non-null   float64
 11  ratings_count   6767 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 638.6+ KB


In [6]:
data.isnull().sum() / len(data) * 100

isbn13             0.000000
isbn10             0.000000
title              0.000000
subtitle          65.036711
authors            1.057269
categories         1.453744
thumbnail          4.831131
description        3.847283
published_year     0.088106
average_rating     0.631424
num_pages          0.631424
ratings_count      0.631424
dtype: float64

In [7]:
rows_to_remove = data[(data["description"].isnull()) | (data["authors"].isnull()) | (data["published_year"].isnull() )| (data["average_rating"].isnull()) |( data["num_pages"].isnull()) | ( data["ratings_count"].isnull())]

In [8]:
len(rows_to_remove) / len(data) * 100 #5.5% of the data

5.3744493392070485

In [9]:
data = data.drop(index=rows_to_remove.index)

In [10]:
(data.isnull().sum() / len(data) * 100).sort_values(ascending=False)

subtitle          64.959652
thumbnail          3.227809
categories         0.512104
isbn13             0.000000
title              0.000000
isbn10             0.000000
authors            0.000000
description        0.000000
published_year     0.000000
average_rating     0.000000
num_pages          0.000000
ratings_count      0.000000
dtype: float64

In [11]:
data["categories"].value_counts() #530 categories is too much, there is something wrong with this column

categories
Fiction                      2510
Juvenile Fiction              521
Biography & Autobiography     390
History                       256
Literary Criticism            163
                             ... 
Humorous stories                1
Ballets                         1
Aged women                      1
Catholic women                  1
Christian fiction               1
Name: count, Length: 530, dtype: int64

In [12]:
data["description"].sample(5) #some desscriptions are too short to be useful and some are too long
# i think characters greater than 25 are better suited for understanding the context

5024    Violence erupts in the poor town of Milagro wh...
3235    FBI Special Agent Dillon Savich teams up with ...
5235    Seventeen-year-old Manhattan society girl Grad...
4516    This is the story of the Tuck family, who are ...
3204    Prejudice, the intricacies of Mediterranean po...
Name: description, dtype: object

In [13]:
data["description_chars"] = data["description"].apply(lambda x: len(x) if isinstance(x, str) else 0)
data = data[data["description_chars"] > 25]

In [14]:
len(data)

6397

In [15]:
# some of the subtitle are missing, so we can have a cojoined title and subtitle, to replace both the title and subtitle
data["title_and_subtitle"] = data["title"].apply(lambda x: x if isinstance(x, str) else "") + " " + data["subtitle"].apply(lambda x: x if isinstance(x, str) else "")

In [16]:
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,description_chars,title_and_subtitle
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,1154,Gilead
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,1200,Spider's Web A Novel
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,109,The One Tree
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,359,Rage of angels
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,295,The Four Loves


In [17]:
data["tagged_description"] = data["isbn13"].apply(str) + " "  + data["description"].apply(str)

In [18]:
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,description_chars,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,1154,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,1200,Spider's Web A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,109,The One Tree,9780006163831 Volume Two of Stephen Donaldson'...
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,359,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,295,The Four Loves,9780006280897 Lewis' work on the nature of lov...


In [19]:
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,description_chars,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,1154,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,1200,Spider's Web A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,109,The One Tree,9780006163831 Volume Two of Stephen Donaldson'...
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,359,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,295,The Four Loves,9780006280897 Lewis' work on the nature of lov...


In [20]:
data = data.drop(columns=[ "title", "subtitle", "description_chars","isbn10"], axis=1)
data.to_csv("books_cleaned.csv", index=False)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6397 entries, 0 to 6809
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   isbn13              6397 non-null   int64  
 1   authors             6397 non-null   object 
 2   categories          6364 non-null   object 
 3   thumbnail           6190 non-null   object 
 4   description         6397 non-null   object 
 5   published_year      6397 non-null   float64
 6   average_rating      6397 non-null   float64
 7   num_pages           6397 non-null   float64
 8   ratings_count       6397 non-null   float64
 9   title_and_subtitle  6397 non-null   object 
 10  tagged_description  6397 non-null   object 
dtypes: float64(4), int64(1), object(6)
memory usage: 599.7+ KB
